In [1]:
#Mount Drive if using colab
from google.colab import drive
drive.mount('/content/drive')

#clone git repository and cd into correct file
!git clone -b IVIM-BRANCH https://github.com/samuelbirchall1/disimpy.git
%cd disimpy
%cd disimpy

#install meshio for mesh reading
!pip install meshio

#import packages
import simulations, substrates, utils, gradients
import meshio
import numpy as np
import matplotlib.pyplot as plt


Mounted at /content/drive
Cloning into 'disimpy'...
remote: Enumerating objects: 929, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 929 (delta 226), reused 218 (delta 181), pack-reused 646
Receiving objects: 100% (929/929), 8.21 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (612/612), done.
/content/disimpy
/content/disimpy/disimpy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.7/167.7 kB 3.7 MB/s eta 0:00:00


In [2]:
#Read mesh and get vertices and faces
mesh = meshio.read(path_to_mesh)
vertices = mesh.points.astype(np.float32)*1e-6 #convert to metres
faces = mesh.cells[0].data

#Create substrate object
substrate = substrates.mesh(
vertices,
faces,
periodic=True)

#Get velocity direction, location
vdir = np.load(path_to_vdir)
vloc = np.load(path_to_vloc)
vloc = vloc*1e-6 #convert to metres
padding=np.zeros(3)
shift = -np.min(vertices, axis=0) + padding
vloc = vloc + shift


Aligning the corner of the simulated voxel with the origin
Moved the vertices by [3.05005096e-06 3.07566688e-06 3.09149505e-06]
Dividing the mesh into subvoxels
Finished dividing the mesh into subvoxels


In [4]:
#set points
rs = gradients.distribute_points(32)

sgp=False

bs = np.linspace(0, 2.5e8, 50)
delta = 20e-3
DELTA = 100e-3
n_t1 = int(2251)
g_1, dt_1, ind = gradients.create_gradient(delta, DELTA, bs, n_t1, rs)
n_t2 = int(3001)
g_2, dt_2, ind = gradients.create_gradient(delta, DELTA, bs, n_t2, rs)
n_t3 = int(3751)
g_3, dt_3, ind = gradients.create_gradient(delta, DELTA, bs, n_t3, rs)

In [5]:
n_walkers = 100000
traj_file = None


flow_velocity = 0.0015 #m/s
flow_length = dt_1*flow_velocity
print(flow_length)
signals, positions, dis = simulations.flow_simulation(
    n_walkers,
    flow_length,
    g_1,
    dt_1,
    substrate,
    traj_file,
    vdir,
    vloc,
)


flow_velocity = 0.0020 #m/s
flow_length = dt_2*flow_velocity
print(flow_length)
signals1, positions1, dis1 = simulations.flow_simulation(
    n_walkers,
    flow_length,
    g_2,
    dt_2,
    substrate,
    traj_file,
    vdir,
    vloc,
)


flow_velocity = 0.0025 #m/s
flow_length = dt_3*flow_velocity
print(flow_length)
signals2, positions2, dis2 = simulations.flow_simulation(
    n_walkers,
    flow_length,
    g_3,
    dt_3,
    substrate,
    traj_file,
    vdir,
    vloc,
)


8.000000000000001e-08
Finished calculating initial positions


KeyboardInterrupt: ignored

In [ ]:
#calculate average signals
avg_s1 = np.mean(signals, axis=0)
avg_s2 = np.mean(signals1, axis=0)
avg_s3 = np.mean(signals2, axis=0)

avg_s = [avg_s1, avg_s2, avg_s3]

In [ ]:
from matplotlib import pyplot as plt
#plot predictions and signals

#calculate analytical signals
v=[0.0015, 0.0020, 0.0025]
l = 24.1 #mean segment length
D_star = (1/6)*l*v
preds = np.exp(-np.array(D_star)*bs)

colours = ['tab:blue', 'tab:orange', 'tab:green']
fig = plt.figure(1, figsize=(6,4))
ax = fig.add_subplot()

for i in range(len(avg_s)):

    ax.scatter(bs, np.log10(avg_s[i]) s=5, c=colours[i], alpha=0.5)
    ax.plot(bs, np.log10(preds[i]), alpha=0.75, c=colours[i], label=f'$v$ = {v[i]*1000} mm/s')

ax.legend(loc='lower left')
ax.set_xlim(0, 2.5e8)

ax.set_ylim(-2.5, 0)
ax.yaxis.set_ticks_position('both')
ax.set_xlabel("b (s/m$^2$)")
ax.set_ylabel("S/S$_0$")
ax.set_yscale('symlog')

fig.tight_layout()

plt.show()

In [ ]:
#sinc regime
delta = 0
DELTA = 5e-4
bs = np.linspace(0, 10e8, 50)
n_t = 1001
#create gradient pulse
g_1, dt_1 = create_SGP_gradient(bs, n_t, DELTA, rs)
#set gradients and dt the same for each experiment
g_2 = g_1
g_3 = g_1
dt_2 = dt_1
dt_3 = dt_1

flow_velocity = 0.0015 #m/s
flow_length = dt_1*flow_velocity
print(flow_length)
signals, positions, dis = simulations.flow_simulation(
    n_walkers,
    flow_length,
    g_1,
    dt_1,
    substrate,
    traj_file,
    vdir,
    vloc,

)


flow_velocity = 0.0020 #m/s
flow_length = dt_2*flow_velocity
print(flow_length)
signals1, positions1, dis1 = simulations.flow_simulation(
    n_walkers,
    flow_length,
    g_2,
    dt_2,
    substrate,
    traj_file,
    vdir,
    vloc,
)


flow_velocity = 0.0025 #m/s
flow_length = dt_3*flow_velocity
print(flow_length)
signals2, positions2, dis2 = simulations.flow_simulation(
    n_walkers,
    flow_length,
    g_3,
    dt_3,
    substrate,
    traj_file,
    vdir,
    vloc,
)

In [ ]:
#plot sinc signals

#new bs for analytical signals
bs1 = np.linspace(0, 10e8, 1000)

#get average signals

avg_s1_sinc = np.mean(signals, axis=0)
avg_s2_sinc = np.mean(signals1, axis=0)
avg_s3_sinc = np.mean(signals2, axis=0)

#average signals
avg_s_sinc = [avg_s1_sinc, avg_s2_sinc, avg_s3_sinc]

#calculate qs
qs = np.sqrt(0.5*bs / DELTA) / (2* np.pi)

#calculate analytic singals

colours = ['tab:blue', 'tab:orange', 'tab:green']
fig = plt.figure(1, figsize=(6,4))
ax = fig.add_subplot()

for i in range(len(avg_s_sinc)):

    ax.scatter(bs, np.log10(avg_s_sinc[i]), s=5, c=colours[i], alpha=0.5)
    ax.plot(bs, np.log10(preds[i]), alpha=0.75, c=colours[i], label=f'$v$ = {v[i]*1000} mm/s')

ax.legend(loc='lower left')
ax.set_xlim(0, 2.5e8)

ax.set_ylim(-2.5, 0)
ax.yaxis.set_ticks_position('both')
ax.set_xlabel("b (s/m$^2$)")
ax.set_ylabel("S/S$_0$")
ax.set_yscale('symlog')

fig.tight_layout()

plt.show()